In [ ]:
import subprocess
print(subprocess.run(['bash', '-lc', 'nvidia-smi || true'], capture_output=True, text=True).stdout)

In [5]:
import subprocess, sys, os, shutil
from pathlib import Path

def pip(*args):
    print(">", *args, flush=True)
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

# 0) Hard reset any prior torch stacks (and possible duplicates under PIP_TARGET)
for pkg in ("torch","torchvision","torchaudio"):
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    "/app/.pip-target/torch",
    "/app/.pip-target/torch-2.8.0.dist-info",
    "/app/.pip-target/torch-2.4.1.dist-info",
    "/app/.pip-target/torchvision",
    "/app/.pip-target/torchvision-0.23.0.dist-info",
    "/app/.pip-target/torchvision-0.19.1.dist-info",
    "/app/.pip-target/torchaudio",
    "/app/.pip-target/torchaudio-2.8.0.dist-info",
    "/app/.pip-target/torchaudio-2.4.1.dist-info",
    "/app/.pip-target/torchgen",
    "/app/.pip-target/functorch",
):
    if os.path.exists(d):
        print("Removing", d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST (matches your CUDA 12.1 container)
pip("install",
    "--index-url", "https://download.pytorch.org/whl/cu121",
    "--extra-index-url", "https://pypi.org/simple",
    "torch==2.4.1", "torchvision==0.19.1", "torchaudio==2.4.1")

# 2) Create a constraints file to freeze torch versions for all later installs
Path("constraints.txt").write_text(
    "torch==2.4.1\n"
    "torchvision==0.19.1\n"
    "torchaudio==2.4.1\n"
)

# 3) Now install NON-torch deps, honoring constraints, and avoid upgrading torch
pip("install", "-c", "constraints.txt",
    "transformers==4.44.2", "accelerate==0.34.2",
    "datasets==2.21.0", "evaluate==0.4.2",
    "sentencepiece", "scikit-learn",
    "--upgrade-strategy", "only-if-needed")

# 4) Sanity gate (hard fail on drift)
import torch
print("torch:", torch.__version__, "built CUDA:", getattr(torch.version, "cuda", None))
print("CUDA available:", torch.cuda.is_available())
assert str(getattr(torch.version,"cuda","")).startswith("12.1"), f"Wrong CUDA build: {torch.version.cuda}"
assert torch.cuda.is_available(), "CUDA not available"
print("GPU:", torch.cuda.get_device_name(0))
print('GPU setup complete!')

Found existing installation: torch 2.4.1+cu121


Uninstalling torch-2.4.1+cu121:
  Successfully uninstalled torch-2.4.1+cu121


Found existing installation: torchvision 0.19.1+cu121
Uninstalling torchvision-0.19.1+cu121:
  Successfully uninstalled torchvision-0.19.1+cu121


Found existing installation: torchaudio 2.4.1+cu121
Uninstalling torchaudio-2.4.1+cu121:
  Successfully uninstalled torchaudio-2.4.1+cu121
> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 59/63 [transformers]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 59/63 [transformers]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 59/63 [transformers]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63/63 [evaluate]


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
GPU setup complete!


In [12]:
import pandas as pd, numpy as np
from sklearn.model_selection import GroupKFold
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset as HFDataset
import random, os, gc
random.seed(42); np.random.seed(42); torch.manual_seed(42)

# Fix tokenizers parallelism before any use
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# CUDA stability
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
torch.cuda.empty_cache()

# Load data
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

# Fallback titles (class_map)
sec_map = {
    'A': 'Human Necessities, Medical, Agriculture',
    'B': 'Operations, Transport, Separation, Mixing',
    'C': 'Chemistry, Metallurgy, Combinatorial Tech',
    'D': 'Textiles, Paper, Building Materials',
    'E': 'Fixed Constructions, Building, Earth Moving',
    'F': 'Mechanical Engineering, Lighting, Heating',
    'G': 'Physics, Computing, Calculating, Counting',
    'H': 'Electricity, Basic Electric Elements',
    'Y': 'Emerging Technologies, Cross-Sectional'
}
class_map = {
    'A61': 'Medical or Veterinary Science; Hygiene',
    'C07': 'Organic Chemistry',
    'C08': 'Organic Macromolecular Compounds; Preparation or Chemical Working-Up',
    'C12': 'Biochemistry; Beer; Spirits; Wine; Vinegar; Microbiology; Enzymology; Mutation or Genetic Engineering',
    'G01': 'Measuring; Testing',
    'G06': 'Computing; Calculating; Counting',
    'H01': 'Basic Electric Elements',
    'H04': 'Electric Communication Technique',
    'B60': 'Vehicles in General',
    'F16': 'Engineering Elements or Units; General Measures for Producing and Maintaining Effective Functioning of Machines or Installations',
    'H05': 'Electric Techniques Not Otherwise Provided For',
    'H03': 'Basic Electronic Circuitry',
    'H01L': 'Semiconductor Devices; Electric Solid State Devices Not Otherwise Provided For',
    'A61K': 'Preparations for Medical, Dental, or Toilet Purposes'
}
def get_title(context):
    cls = context[:3] if len(context) >= 3 else context[0]
    sec = context[0]
    return class_map.get(cls, sec_map.get(sec, 'no title'))
train_df['title'] = train_df['context'].apply(get_title)
test_df['title']  = test_df['context'].apply(get_title)

# Hierarchical tokens
all_ctx = pd.concat([train_df['context'], test_df['context']]).unique()
def get_hier_parts(code):
    sec = code[0] if code else 'X'
    cls = code[:3] if len(code) >= 3 else sec
    sub = code[:4] if len(code) >= 4 else cls
    grp = code[:7].replace('/', '') if len(code) >= 7 else sub
    return f"<SEC_{sec}>", f"<CLS_{cls}>", f"<SUB_{sub}>", f"<GRP_{grp}>"
special_tokens = set()
for c in all_ctx:
    special_tokens.update(get_hier_parts(c))

model_name = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
n_added = tokenizer.add_special_tokens({'additional_special_tokens': sorted(special_tokens)})
print(f'Added {n_added} hierarchical tokens. Vocab: {len(tokenizer)}')

def hier_str(code):
    return ' '.join(get_hier_parts(code)) + ' '

def prepare_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f'{hh}anchor: {a} [CPC] {c} {t}' for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f'{hh}target: {b} [CPC] {c} {t}' for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=128, truncation=True, padding=False)
    if 'score' in examples:
        mi['labels'] = examples['score']
    return mi

def prepare_test_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f'{hh}anchor: {a} [CPC] {c} {t}' for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f'{hh}target: {b} [CPC] {c} {t}' for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=128, truncation=True, padding=False)
    return mi

# Single group-aware split (~80/20)
groups = train_df['anchor'].values
unique_groups = np.unique(groups)
np.random.seed(42)
val_size = len(unique_groups) // 5
val_groups = np.random.choice(unique_groups, size=val_size, replace=False)
va_mask = np.isin(groups, val_groups)
va_idx = np.where(va_mask)[0]
tr_idx = np.where(~va_mask)[0]
print(f'Single fold: train {len(tr_idx)}, val {len(va_idx)}')

oof_large = np.full(len(train_df), np.nan)
test_preds_large = np.zeros(len(test_df))
fold_pearsons_large = []

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

print(f'\n--- Large Fold 1/1 ---')
print(f'Fold 1 start time: {pd.Timestamp.now()}')
tr = train_df.iloc[tr_idx]
va = train_df.iloc[va_idx]

ds_tr = HFDataset.from_pandas(tr[['anchor','target','context','title','score']])
ds_va = HFDataset.from_pandas(va[['anchor','target','context','title','score']])
ds_te = HFDataset.from_pandas(test_df[['anchor','target','context','title']])

ds_tr = ds_tr.map(prepare_features, batched=True, remove_columns=ds_tr.column_names)
ds_va = ds_va.map(prepare_features, batched=True, remove_columns=ds_va.column_names)
ds_te = ds_te.map(prepare_test_features, batched=True, remove_columns=ds_te.column_names)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, problem_type='regression')
model.gradient_checkpointing_enable()
model.config.use_cache = False
if n_added:
    model.resize_token_embeddings(len(tokenizer))

args = TrainingArguments(
    output_dir='./fold_large_0',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    logging_steps=20,
    evaluation_strategy='no',
    save_strategy='no',
    load_best_model_at_end=False,
    fp16=False,
    bf16=True,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    report_to=[],
    disable_tqdm=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tr,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()
print(f'Fold 1 end time: {pd.Timestamp.now()}')

va_pred = np.clip(trainer.predict(ds_va).predictions.flatten(), 0, 1)
oof_large[va_idx] = va_pred
fold_pearson = pearsonr(va_pred, va['score'].values)[0]
fold_pearsons_large.append(fold_pearson)
print(f'Large Fold 1 Pearson: {fold_pearson:.4f}')

test_preds_large = np.clip(trainer.predict(ds_te).predictions.flatten(), 0, 1)

# Cleanup
del model, trainer, ds_tr, ds_va, ds_te
gc.collect()
torch.cuda.empty_cache()

valid_mask = ~np.isnan(oof_large)
oof_pearson_large = pearsonr(oof_large[valid_mask], train_df['score'][valid_mask].values)[0]
print(f'\nPartial OOF Pearson (large 1f): {oof_pearson_large:.4f}')
print(f'Mean fold Pearson (large): {np.mean(fold_pearsons_large):.4f} +/- {np.std(fold_pearsons_large):.4f}')

iso_large = IsotonicRegression(out_of_bounds='clip')
iso_large.fit(oof_large[valid_mask], train_df['score'][valid_mask].values)
oof_cal_large = iso_large.predict(oof_large[valid_mask])
oof_cal_pearson_large = pearsonr(oof_cal_large, train_df['score'][valid_mask].values)[0]
print(f'Calibrated Partial OOF Pearson (large): {oof_cal_pearson_large:.4f}')

test_final_large = np.clip(iso_large.predict(test_preds_large) if oof_cal_pearson_large > oof_pearson_large else test_preds_large, 0, 1)

np.save('oof_large.npy', oof_large)
np.save('test_large.npy', test_preds_large)
print('Saved large model partial OOF (NaNs outside val) and test predictions to .npy files')

/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Added 326 hierarchical tokens. Vocab: 128327
Single fold: train 26053, val 6772

--- Large Fold 1/1 ---
Fold 1 start time: 2025-09-27 14:45:11.816562


Map:   0%|          | 0/26053 [00:00<?, ? examples/s]

Map:   8%|▊         | 2000/26053 [00:00<00:01, 12565.81 examples/s]

Map:  15%|█▌        | 4000/26053 [00:00<00:01, 13316.26 examples/s]

...

/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss


Fold 1 end time: 2025-09-27 14:54:58.661373


Large Fold 1 Pearson: 0.8412



Partial OOF Pearson (large 1f): 0.8412
Mean fold Pearson (large): 0.8412 +/- 0.0000
Calibrated Partial OOF Pearson (large): 0.8455
Saved large model partial OOF (NaNs outside val) and test predictions to .npy files


In [13]:
import pandas as pd, numpy as np
from sklearn.model_selection import GroupKFold
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset as HFDataset
import random, os, gc
random.seed(42); np.random.seed(42); torch.manual_seed(42)

# Fix tokenizers parallelism before any use
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# CUDA stability
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
torch.cuda.empty_cache()

# Load data
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

# Fallback titles (class_map)
sec_map = {
    'A': 'Human Necessities, Medical, Agriculture',
    'B': 'Operations, Transport, Separation, Mixing',
    'C': 'Chemistry, Metallurgy, Combinatorial Tech',
    'D': 'Textiles, Paper, Building Materials',
    'E': 'Fixed Constructions, Building, Earth Moving',
    'F': 'Mechanical Engineering, Lighting, Heating',
    'G': 'Physics, Computing, Calculating, Counting',
    'H': 'Electricity, Basic Electric Elements',
    'Y': 'Emerging Technologies, Cross-Sectional'
}
class_map = {
    'A61': 'Medical or Veterinary Science; Hygiene',
    'C07': 'Organic Chemistry',
    'C08': 'Organic Macromolecular Compounds; Preparation or Chemical Working-Up',
    'C12': 'Biochemistry; Beer; Spirits; Wine; Vinegar; Microbiology; Enzymology; Mutation or Genetic Engineering',
    'G01': 'Measuring; Testing',
    'G06': 'Computing; Calculating; Counting',
    'H01': 'Basic Electric Elements',
    'H04': 'Electric Communication Technique',
    'B60': 'Vehicles in General',
    'F16': 'Engineering Elements or Units; General Measures for Producing and Maintaining Effective Functioning of Machines or Installations',
    'H05': 'Electric Techniques Not Otherwise Provided For',
    'H03': 'Basic Electronic Circuitry',
    'H01L': 'Semiconductor Devices; Electric Solid State Devices Not Otherwise Provided For',
    'A61K': 'Preparations for Medical, Dental, or Toilet Purposes'
}
def get_title(context):
    cls = context[:3] if len(context) >= 3 else context[0]
    sec = context[0]
    return class_map.get(cls, sec_map.get(sec, 'no title'))
train_df['title'] = train_df['context'].apply(get_title)
test_df['title']  = test_df['context'].apply(get_title)

# Hierarchical tokens
all_ctx = pd.concat([train_df['context'], test_df['context']]).unique()
def get_hier_parts(code):
    sec = code[0] if code else 'X'
    cls = code[:3] if len(code) >= 3 else sec
    sub = code[:4] if len(code) >= 4 else cls
    grp = code[:7].replace('/', '') if len(code) >= 7 else sub
    return f"<SEC_{sec}>", f"<CLS_{cls}>", f"<SUB_{sub}>", f"<GRP_{grp}>"
special_tokens = set()
for c in all_ctx:
    special_tokens.update(get_hier_parts(c))

model_name = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
n_added = tokenizer.add_special_tokens({'additional_special_tokens': sorted(special_tokens)})
print(f'Added {n_added} hierarchical tokens. Vocab: {len(tokenizer)}')

def hier_str(code):
    return ' '.join(get_hier_parts(code)) + ' '

def prepare_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f'{hh}anchor: {a} [CPC] {c} {t}' for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f'{hh}target: {b} [CPC] {c} {t}' for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=192, truncation=True, padding=False)
    if 'score' in examples:
        mi['labels'] = examples['score']
    return mi

def prepare_test_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f'{hh}anchor: {a} [CPC] {c} {t}' for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f'{hh}target: {b} [CPC] {c} {t}' for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=192, truncation=True, padding=False)
    return mi

# 2-fold GroupKFold
groups = train_df['anchor'].values
gkf = GroupKFold(n_splits=2)
oof_large_2f = np.full(len(train_df), np.nan)
test_preds_large_2f = np.zeros(len(test_df))
fold_pearsons_large_2f = []

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

for fold, (tr_idx, va_idx) in enumerate(gkf.split(train_df, groups=groups)):
    print(f'\n--- Large Fold {fold+1}/2 ---')
    print(f'Fold {fold+1} start time: {pd.Timestamp.now()}')
    tr = train_df.iloc[tr_idx]
    va = train_df.iloc[va_idx]

    ds_tr = HFDataset.from_pandas(tr[['anchor','target','context','title','score']])
    ds_va = HFDataset.from_pandas(va[['anchor','target','context','title','score']])
    ds_te = HFDataset.from_pandas(test_df[['anchor','target','context','title']])

    ds_tr = ds_tr.map(prepare_features, batched=True, remove_columns=ds_tr.column_names)
    ds_va = ds_va.map(prepare_features, batched=True, remove_columns=ds_va.column_names)
    ds_te = ds_te.map(prepare_test_features, batched=True, remove_columns=ds_te.column_names)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, problem_type='regression')
    model.gradient_checkpointing_enable()
    model.config.use_cache = False
    if n_added:
        model.resize_token_embeddings(len(tokenizer))

    args = TrainingArguments(
        output_dir=f'./fold_large_2f_{fold}',
        num_train_epochs=1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=64,
        gradient_accumulation_steps=2,
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_ratio=0.1,
        lr_scheduler_type='cosine',
        logging_steps=20,
        evaluation_strategy='no',
        save_strategy='no',
        load_best_model_at_end=False,
        fp16=False,
        bf16=True,
        dataloader_num_workers=0,
        dataloader_pin_memory=True,
        report_to=[],
        disable_tqdm=False,
        remove_unused_columns=False,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tr,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    trainer.train()
    print(f'Fold {fold+1} end time: {pd.Timestamp.now()}')

    va_pred = np.clip(trainer.predict(ds_va).predictions.flatten(), 0, 1)
    oof_large_2f[va_idx] = va_pred
    fold_pearson = pearsonr(va_pred, va['score'].values)[0]
    fold_pearsons_large_2f.append(fold_pearson)
    print(f'Large Fold {fold+1} Pearson: {fold_pearson:.4f}')

    te_pred = np.clip(trainer.predict(ds_te).predictions.flatten(), 0, 1)
    test_preds_large_2f += te_pred / 2  # average over folds

    # Cleanup
    del model, trainer, ds_tr, ds_va, ds_te
    gc.collect()
    torch.cuda.empty_cache()

valid_mask_2f = ~np.isnan(oof_large_2f)
oof_pearson_large_2f = pearsonr(oof_large_2f[valid_mask_2f], train_df['score'][valid_mask_2f].values)[0]
print(f'\nPartial OOF Pearson (large 2f): {oof_pearson_large_2f:.4f}')
print(f'Mean fold Pearson (large 2f): {np.mean(fold_pearsons_large_2f):.4f} +/- {np.std(fold_pearsons_large_2f):.4f}')

iso_large_2f = IsotonicRegression(out_of_bounds='clip')
iso_large_2f.fit(oof_large_2f[valid_mask_2f], train_df['score'][valid_mask_2f].values)
oof_cal_large_2f = iso_large_2f.predict(oof_large_2f[valid_mask_2f])
oof_cal_pearson_large_2f = pearsonr(oof_cal_large_2f, train_df['score'][valid_mask_2f].values)[0]
print(f'Calibrated Partial OOF Pearson (large 2f): {oof_cal_pearson_large_2f:.4f}')

test_final_large_2f = np.clip(iso_large_2f.predict(test_preds_large_2f) if oof_cal_pearson_large_2f > oof_pearson_large_2f else test_preds_large_2f, 0, 1)

np.save('oof_large_2f.npy', oof_large_2f)
np.save('test_large_2f.npy', test_preds_large_2f)
print('Saved large 2f partial OOF and averaged test predictions to .npy files')

/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Added 326 hierarchical tokens. Vocab: 128327

--- Large Fold 1/2 ---
Fold 1 start time: 2025-09-27 15:02:02.111880


Map:   0%|          | 0/16412 [00:00<?, ? examples/s]

Map:  12%|█▏        | 2000/16412 [00:00<00:01, 12883.07 examples/s]

Map:  24%|██▍       | 4000/16412 [00:00<00:00, 13403.26 examples/s]

...

/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss


Fold 2 end time: 2025-09-27 15:15:07.619707


Large Fold 2 Pearson: -0.0036



Partial OOF Pearson (large 2f): 0.5723
Mean fold Pearson (large 2f): 0.4036 +/- 0.4071
Calibrated Partial OOF Pearson (large 2f): 0.5742
Saved large 2f partial OOF and averaged test predictions to .npy files


In [3]:
import pandas as pd, numpy as np
from sklearn.model_selection import GroupKFold
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
import torch, importlib.metadata
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset as HFDataset
import random, os
random.seed(42); np.random.seed(42); torch.manual_seed(42)

# CUDA stability flags
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
torch.cuda.empty_cache()

# Load data
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

# Fallback titles (class_map as before)
sec_map = {
    'A': 'Human Necessities, Medical, Agriculture',
    'B': 'Operations, Transport, Separation, Mixing',
    'C': 'Chemistry, Metallurgy, Combinatorial Tech',
    'D': 'Textiles, Paper, Building Materials',
    'E': 'Fixed Constructions, Building, Earth Moving',
    'F': 'Mechanical Engineering, Lighting, Heating',
    'G': 'Physics, Computing, Calculating, Counting',
    'H': 'Electricity, Basic Electric Elements',
    'Y': 'Emerging Technologies, Cross-Sectional'
}
class_map = {
    'A61': 'Medical or Veterinary Science; Hygiene',
    'C07': 'Organic Chemistry',
    'C08': 'Organic Macromolecular Compounds; Preparation or Chemical Working-Up',
    'C12': 'Biochemistry; Beer; Spirits; Wine; Vinegar; Microbiology; Enzymology; Mutation or Genetic Engineering',
    'G01': 'Measuring; Testing',
    'G06': 'Computing; Calculating; Counting',
    'H01': 'Basic Electric Elements',
    'H04': 'Electric Communication Technique',
    'B60': 'Vehicles in General',
    'F16': 'Engineering Elements or Units; General Measures for Producing and Maintaining Effective Functioning of Machines or Installations',
    'H05': 'Electric Techniques Not Otherwise Provided For',
    'H03': 'Basic Electronic Circuitry',
    'H01L': 'Semiconductor Devices; Electric Solid State Devices Not Otherwise Provided For',
    'A61K': 'Preparations for Medical, Dental, or Toilet Purposes'
}
def get_title(context):
    cls = context[:3] if len(context) >= 3 else context[0]
    sec = context[0]
    return class_map.get(cls, sec_map.get(sec, 'no title'))
train_df['title'] = train_df['context'].apply(get_title)
test_df['title']  = test_df['context'].apply(get_title)

# Build hierarchical CPC tokens from train+test
all_ctx = pd.concat([train_df['context'], test_df['context']]).unique()

def get_hier_parts(code):
    sec = code[0] if code else 'X'
    cls = code[:3] if len(code) >= 3 else sec
    sub = code[:4] if len(code) >= 4 else cls
    grp = code[:7].replace('/', '') if len(code) >= 7 else sub  # e.g., A61K31
    return f"<SEC_{sec}>", f"<CLS_{cls}>", f"<SUB_{sub}>", f"<GRP_{grp}>"

special_tokens = set()
for c in all_ctx:
    special_tokens.update(get_hier_parts(c))

model_name = 'microsoft/deberta-v3-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
n_added = tokenizer.add_special_tokens({'additional_special_tokens': sorted(special_tokens)})
print(f"Added {n_added} hierarchical tokens. Vocab: {len(tokenizer)}")

def hier_str(code):
    return " ".join(get_hier_parts(code)) + " "

def prepare_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f"{hh}anchor: {a} [CPC] {c} {t}" for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f"{hh}target: {b} [CPC] {c} {t}" for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=192, truncation=True, padding=False)
    if 'score' in examples:
        mi['labels'] = examples['score']
    return mi

def prepare_test_features(examples):
    h = [hier_str(c) for c in examples['context']]
    t1 = [f"{hh}anchor: {a} [CPC] {c} {t}" for hh,a,c,t in zip(h, examples['anchor'], examples['context'], examples['title'])]
    t2 = [f"{hh}target: {b} [CPC] {c} {t}" for hh,b,c,t in zip(h, examples['target'], examples['context'], examples['title'])]
    mi = tokenizer(t1, t2, max_length=192, truncation=True, padding=False)
    return mi

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.clip(preds.flatten(), 0, 1)
    return {'pearson': pearsonr(preds, labels)[0]}

gkf = GroupKFold(n_splits=5)  # Use 5 folds as recommended
oof_large = np.zeros(len(train_df))
test_preds_large = np.zeros((5, len(test_df)))
fold_pearsons_large = []

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

for fold, (tr_idx, va_idx) in enumerate(gkf.split(train_df, groups=train_df['anchor'])):
    print(f'\n--- Large Fold {fold+1}/5 ---')
    tr = train_df.iloc[tr_idx]
    va = train_df.iloc[va_idx]

    ds_tr = HFDataset.from_pandas(tr[['anchor','target','context','title','score']])
    ds_va = HFDataset.from_pandas(va[['anchor','target','context','title','score']])
    ds_te = HFDataset.from_pandas(test_df[['anchor','target','context','title']])

    ds_tr = ds_tr.map(prepare_features, batched=True, remove_columns=ds_tr.column_names)
    ds_va = ds_va.map(prepare_features, batched=True, remove_columns=ds_va.column_names)
    ds_te = ds_te.map(prepare_test_features, batched=True, remove_columns=ds_te.column_names)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, problem_type='regression')
    model.gradient_checkpointing_enable()
    model.config.use_cache = False
    if n_added:
        model.resize_token_embeddings(len(tokenizer))

    args = TrainingArguments(
        output_dir=f'./fold_large_{fold}',
        num_train_epochs=3,
        per_device_train_batch_size=4,  # Safer start
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=4,  # effective 16
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_ratio=0.15,
        lr_scheduler_type='cosine',
        logging_steps=50,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='pearson',
        greater_is_better=True,
        save_total_limit=1,
        fp16=False,
        bf16=True,  # Ampere stable
        dataloader_num_workers=2,
        dataloader_pin_memory=False,
        eval_accumulation_steps=32,
        report_to=None,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tr,
        eval_dataset=ds_va,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=2e-4)],
    )
    trainer.train()

    def predict_tta(trainer, dataset, df, is_test=False):
        normal = trainer.predict(dataset).predictions.flatten()
        swapped_df = df.copy()
        swapped_df[['anchor','target']] = swapped_df[['target','anchor']]
        swapped = HFDataset.from_pandas(swapped_df[['anchor','target','context','title']] if is_test else swapped_df[['anchor','target','context','title','score']])
        swapped = swapped.map(prepare_test_features if is_test else prepare_features, batched=True, remove_columns=swapped.column_names)
        swapped_pred = trainer.predict(swapped).predictions.flatten()
        return np.clip((normal + swapped_pred)/2, 0, 1)

    va_pred = predict_tta(trainer, ds_va, va, is_test=False)
    oof_large[va_idx] = va_pred
    fold_pearson = pearsonr(va_pred, va['score'].values)[0]
    fold_pearsons_large.append(fold_pearson)
    print(f'Large Fold {fold+1} Pearson: {fold_pearson:.4f}')

    test_preds_large[fold] = predict_tta(trainer, ds_te, test_df, is_test=True)

oof_pearson_large = pearsonr(oof_large, train_df['score'].values)[0]
print(f'\nFull OOF Pearson (large): {oof_pearson_large:.4f}')
print(f'Mean fold Pearson (large): {np.mean(fold_pearsons_large):.4f} +/- {np.std(fold_pearsons_large):.4f}')

iso_large = IsotonicRegression(out_of_bounds='clip')
iso_large.fit(oof_large, train_df['score'].values)
oof_cal_large = iso_large.predict(oof_large)
oof_cal_pearson_large = pearsonr(oof_cal_large, train_df['score'].values)[0]
print(f'Calibrated OOF Pearson (large): {oof_cal_pearson_large:.4f}')

test_avg_large = np.mean(test_preds_large, axis=0)
test_final_large = np.clip(iso_large.predict(test_avg_large) if oof_cal_pearson_large > oof_pearson_large else test_avg_large, 0, 1)

np.save('oof_large.npy', oof_large)
np.save('test_large.npy', test_avg_large)
print('Saved large model OOF and test predictions to .npy files')

/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/app/.pip-target/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Added 326 hierarchical tokens. Vocab: 128327

--- Large Fold 1/5 ---


Map:   0%|          | 0/26260 [00:00<?, ? examples/s]

Map:   8%|▊         | 2000/26260 [00:00<00:01, 16821.32 examples/s]

Map:  15%|█▌        | 4000/26260 [00:00<00:01, 18145.97 examples/s]

...

/app/.pip-target/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/app/.pip-target/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss


In [ ]:
import sys, subprocess, shutil, glob
for p in glob.glob('/app/.pip-target/accelerate*'):
    print('Removing', p); shutil.rmtree(p, ignore_errors=True)
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "accelerate"], check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-c", "constraints.txt",
                "--no-cache-dir", "--force-reinstall", "accelerate==0.30.1"], check=True)
import accelerate; print("accelerate:", accelerate.__version__)

In [2]:
import torch
from transformers import AutoModelForSequenceClassification
m = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-large', num_labels=1).cuda()
x = { 'input_ids': torch.randint(0, 1000, (4, 192), device='cuda'),
      'attention_mask': torch.ones(4, 192, device='cuda') }
with torch.autocast('cuda', dtype=torch.bfloat16):
    y = m(**x)
print('✓ Forward OK'); del m, x, y; torch.cuda.empty_cache()

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Forward OK
